In [3]:
print("berger")

berger


In [4]:
%pwd

'd:\\Work\\C_Drive\\UBotz\\research'

In [5]:
import os
os.chdir('../')

In [6]:
pwd

'd:\\Work\\C_Drive\\UBotz'

In [7]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [8]:
def load_pdf_file(data):
    loader = DirectoryLoader(data,glob='*.pdf',loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [9]:
extracted_data = load_pdf_file('data/')

In [10]:
# extracted_data

In [11]:
def text_splitter(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [12]:
text_chunks = text_splitter(extracted_data)
print("Length of Text chunks:", len(text_chunks))

Length of Text chunks: 7063


In [13]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

def download_hugging_face_embeddings():
    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2") #384 dimension vector
    return embeddings

In [14]:
embeddings = download_hugging_face_embeddings()

C:\Users\GOD\AppData\Local\Temp\ipykernel_1156\1468717217.py:4: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2") #384 dimension vector


In [15]:
query_result = embeddings.embed_query("Hellow World")
print("length of query result:", len(query_result))

length of query result: 384


In [16]:
# query_result

In [2]:
import pinecone

In [17]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=os.environ.get('PINECONE_API_KEY'))

index_name = "ubotz"
pc.create_index(name=index_name,
                dimension=384,
                metric="cosine",
                spec=ServerlessSpec(cloud='aws', region='us-east-1'))

{
    "name": "ubotz",
    "metric": "cosine",
    "host": "ubotz-585ab1f.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [20]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
    )

In [21]:
# load existing index
docsearch = PineconeVectorStore.from_existing_index(index_name, embeddings)
docsearch

In [22]:
retriever = docsearch.as_retriever(search_type="similarity",search_kwargs={"k": 3})

In [23]:
retriever_docs = retriever.invoke("Who decides repo rate, reverse repo rate and bank rate ?")

In [24]:
retriever_docs

[Document(id='08c4e0ff-769f-4fcb-85c2-b4d68c614dd1', metadata={'creationdate': '', 'creator': 'PyPDF', 'moddate': '2022-02-11T03:28:52+00:00', 'page': 384.0, 'page_label': '385', 'producer': 'PSPDFKit', 'source': 'data\\Pinnacle GS Eng Updated.pdf', 'title': 'Chapter-wise Index CA 2021-22 Book (English)', 'total_pages': 627.0}, page_content='On  10  February  1937,  he  founded  the \n Indian  Overseas  Bank,  With  the \n initiative  to  improve  the  country’s \n industrial  sector.  It  is  headquartered  in \n Chennai, India. \n Q.  152.Who  decides  repo  rate,  reverse \n repo rate and bank rate ? \n SSC MTS 13/08/2019 (Afternoon) \n (a) SEBI \n (b) NABARD \n (c) Reserve Bank of India \n (d) SBI \n Sol.  (c)  Reserve  Bank  of  India  ,  The \n monetary  policies  are  regulated  by  the \n Reserve  bank  of  India.  These  policies'),
 Document(id='bb80153c-08e1-4d00-8cb6-d3857a4b8d14', metadata={'creationdate': '', 'creator': 'PyPDF', 'moddate': '2022-02-11T03:28:52+00:00', 'pa

In [31]:
GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    api_key="xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
    temperature=0.4,
    max_tokens=500,
)

In [39]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are a helpful assistant that answers questions based on the provided context. "
    "use the following pieces of rerieved content to answer the question: "
    "If you don't know the answer, just say that you don't know. "
    "Do not make up an answer."
    "answer concise. "
    "context: {context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

In [40]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain= create_retrieval_chain(retriever, question_answer_chain)

In [41]:
response = rag_chain.invoke({"input": "Who decides repo rate, reverse repo rate and bank rate ?"})
print(response["answer"])

Reserve Bank of India decides repo rate, reverse repo rate and bank rate.


In [42]:
response = rag_chain.invoke({"input": "What is acne ?"})
print(response["answer"])

I'm sorry, but based on the context provided, I cannot answer what acne is.


In [ ]:
response = rag_chain.invoke({"input": "As of May 2019, who among the following has won the maximum number of Grand Slam Tournaments in Men's Tennis?"})
print(response["answer"])

As of May 2019, Roger Federer had won the maximum number of Grand Slam Tournaments in Men's Tennis.
